In [ ]:
print("TEST")

# NOTE: This notebook is trained locally on my HP PAVILION laptop running UBUNTU subbsystems.
# incase you wish to execute this notebook, please use a ubuntu system with nvidia rtx card and nvidea cuda enabled.
# or use google colab with T4 gpu. make changes to trainer save and other paths repos accordingly
# Siddhant Mahajan, Group 20@ VITB::Project exhibition 2

TEST


# We will use a pre-trained model "DistilBERT" by hugging Face

Sources and citations:

https://github.com/huggingface/transformers/tree/main/examples/research_projects/distillation

https://huggingface.co/docs/transformers/model_doc/distilbert

https://arxiv.org/abs/1910.01108


In [2]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Lets check if GPU is available from torch.

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
# !pip install datasets transformers[torch]
# !pip install accelerate -U

# We will import the IMDB datasets. it has over 25000 labelled data
First we will create a small sample of dataset to check our training and testing results

In [5]:
from datasets import load_dataset
imdb = load_dataset("imdb")

/home/sidthechillguy/miniconda3/envs/deeplearn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating unsupervised split: 100%|███████████████████████████████████| 50000/50000 [00:00<00:00, 272876.22 examples/s]


In [7]:
mini_train_dataset = imdb["train"].shuffle(seed=15).select([i for i in list(range(6000))])
mini_test_dataset = imdb["test"].shuffle(seed=15).select([i for i in list(range(600))])


We will import the distilbert pre-trained model from huggingface

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


We create a mapped dataset wi=hich has tokenised text input for further usage

In [9]:
def preprocess_function(text_in):
   return tokenizer(text_in["text"], truncation=True)

tokenized_train = mini_train_dataset.map(preprocess_function, batched=True)
tokenized_test = mini_test_dataset.map(preprocess_function, batched=True)


Map: 100%|███████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 4984.55 examples/s]


We will apply padding to textual inputs to maintain the shape formats.
Padding picks the longest sentence in the datset and appends padding tokens (usually defined in the vocabulary of the model/tokeniser we use, example: 0 is a padding sequence, so any amount of 0 will be a filler) to shorter sentences to even out.

In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-03-18 21:27:34.920621: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 21:27:34.920710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 21:27:35.326510: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 21:27:36.686690: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 21:27:42.105028: W tensorflow/compiler/tf2

Preprocessing is now compelete. We will statrt performing training

In [11]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [17]:
from transformers import TrainingArguments, Trainer

output_dir = '/home/sidthechillguy/project_exhibition_2/distilbert_outs/'

training_args = TrainingArguments(
   output_dir=output_dir,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [18]:
trainer.train()


Step,Training Loss
500,0.303500


TrainOutput(global_step=750, training_loss=0.26368716939290365, metrics={'train_runtime': 5181.3602, 'train_samples_per_second': 2.316, 'train_steps_per_second': 0.145, 'total_flos': 1572706771724736.0, 'train_loss': 0.26368716939290365, 'epoch': 2.0})

In [19]:
trainer.evaluate()


/tmp/ipykernel_121/1345777986.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/home/sidthechillguy/miniconda3/envs/deeplearn/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ConnectionError: Couldn't reach https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py (ConnectionError(MaxRetryError("HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd3d62a7df0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))")))

In [21]:
trainer.save_model('/home/sidthechillguy/project_exhibition_2/distilbert_outs/test')

# Load the saved model

Label 0 is negetive. Label 1 is postive.

In [2]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification

repo = '/home/sidthechillguy/project_exhibition_2/distilbert_outs/model1'

pre_model = AutoModelForSequenceClassification.from_pretrained(repo)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

sentiment_model = pipeline("sentiment-analysis",model=pre_model, tokenizer = tokenizer)
sentiment_model(["I love this move", "This movie sucks!"])


/home/sidthechillguy/miniconda3/envs/deeplearn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-19 13:20:07.952420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 13:20:07.952568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 13:20:08.380362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 13:20:09.673508: I tensorflow/core/platform/cpu_feature_guard.cc:182] Thi

[{'label': 'LABEL_1', 'score': 0.978310227394104},
 {'label': 'LABEL_0', 'score': 0.9821276664733887}]

In [9]:
sentiment_model(["Today was very hard. I had to carry laptop all day. My back hurts.",
                 "My day was busy and completed desired task.",
                 "Today was a very hard day. I had seen my friend in a very critical health condition.",
                 "Today I woke very late due to which I was not able to attend today's lecture and in the afternoon we celebrated birthday of my friend and food of my hostel mess was good",
                 "It was okay. I had to buy a case study which I did not want",
                 "today was'nt a good day as i am suffering from a severe cold and cough I couldnt study well for my maths exam which is on march 20",
                 "Today was very hard. I attended my lecture and used my laptop to do some work. then i met my friend and he asked me to help in his project so i helped him to fix some errors in his project Thats why i have to carry my laptop all day because of that my back hurts but i'm happy because i did some work and helped my friend",
                 "Today I am feeling slightly unpleasant which best describes my loneliness, disappointment, embarrassed. It was caused by eductation, hobbies and self care."])

#Labels:
#Example, negetive
#pappa, positive
#anirudh, negetive
#ashutosh, positive
#suhani, negetive
#abhijay, negetive
#sarvesh, positive
#bidhan, negetive

[{'label': 'LABEL_0', 'score': 0.9106560349464417},
 {'label': 'LABEL_1', 'score': 0.7430962324142456},
 {'label': 'LABEL_1', 'score': 0.8099514245986938},
 {'label': 'LABEL_1', 'score': 0.5934730768203735},
 {'label': 'LABEL_0', 'score': 0.7771883606910706},
 {'label': 'LABEL_1', 'score': 0.5629598498344421},
 {'label': 'LABEL_1', 'score': 0.9225858449935913},
 {'label': 'LABEL_0', 'score': 0.9074738025665283}]

In [ ]:
# Correct classification:
#Negetive: 2/4
# positive: 4/4

# incorrect:
# positive: 0
# negetive: 2